In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f493e229850>)

In [4]:
!ls -l ted_en-20160408.xml

-rw-r--r-- 1 root root 74533638 Sep  3 04:17 ted_en-20160408.xml


## 훈련 데이터 전처리하기

In [5]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

In [6]:
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [7]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [8]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [9]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [10]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [11]:
print(f'총 샘플의 개수 : {len(result)}')

총 샘플의 개수 : 273424


In [12]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


여기서 Word2Vec의 하이퍼파라미터값은 다음과 같습니다.
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 - 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [13]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [14]:
model.wv.most_similar('man')

[('woman', 0.8608222603797913),
 ('guy', 0.8056197762489319),
 ('lady', 0.7870907783508301),
 ('boy', 0.777847409248352),
 ('girl', 0.7661465406417847),
 ('gentleman', 0.73875892162323),
 ('poet', 0.7282140254974365),
 ('soldier', 0.719333827495575),
 ('kid', 0.7037489414215088),
 ('friend', 0.6914616823196411)]

In [17]:
# man - boy + girl => woman?
model.wv.most_similar(positive=['man','girl'], negative=['boy'], topn=3)

[('woman', 0.8670567870140076),
 ('lady', 0.7674709558486938),
 ('guy', 0.7533614039421082)]

In [26]:
model.most_similar(positive=['man','girl'], negative=['boy'], topn=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.8670567870140076),
 ('lady', 0.7674709558486938),
 ('guy', 0.7533614039421082)]

## 모델 저장하고 로드하기

In [18]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v')

In [20]:
!ls -l

total 97912
-rw-r--r-- 1 root root 25722750 Sep  3 04:43 eng_w2v
drwxr-xr-x 1 root root     4096 Sep  1 19:26 sample_data
-rw-r--r-- 1 root root 74533638 Sep  3 04:17 ted_en-20160408.xml


In [21]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델로드

In [23]:
loaded_model.most_similar('man')

[('woman', 0.8608222603797913),
 ('guy', 0.8056197762489319),
 ('lady', 0.7870907783508301),
 ('boy', 0.777847409248352),
 ('girl', 0.7661465406417847),
 ('gentleman', 0.73875892162323),
 ('poet', 0.7282140254974365),
 ('soldier', 0.719333827495575),
 ('kid', 0.7037489414215088),
 ('friend', 0.6914616823196411)]

In [25]:
loaded_model.wv.most_similar('man')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.8608222603797913),
 ('guy', 0.8056197762489319),
 ('lady', 0.7870907783508301),
 ('boy', 0.777847409248352),
 ('girl', 0.7661465406417847),
 ('gentleman', 0.73875892162323),
 ('poet', 0.7282140254974365),
 ('soldier', 0.719333827495575),
 ('kid', 0.7037489414215088),
 ('friend', 0.6914616823196411)]